In [1]:
import torch.nn as nn
from torchvision import models
import torch
import torchvision.transforms as transforms
from PIL import Image
from transformers import AutoTokenizer, AutoModel
import pandas as pd

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")  # GPUデバイスを取得
else:
    device = torch.device("cpu")  # CPUデバイスを取得

In [3]:
import importlib
import Entity
from Entity import CustomDataset
# importlib.reload(learnign)


NameError: name 'learnign' is not defined

In [4]:
from learning.Entity.CustomDataset import PokemonMoveDataset


dataset = PokemonMoveDataset('D:/tanaka/Documents/poke-move/data/my-dataset/dataset.txt')

In [5]:
from torch.utils.data import DataLoader
learning_rate = 1e-5
batch_size = 16
epochs = 5
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, test_dataset = torch.utils.data.random_split(
    dataset, [train_size, val_size]
)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)

In [6]:
import torch.nn.functional as F
import sys
import os
from datetime import datetime
from learning.Encoder.MoveEncoder import MoveEncoder
from learning.Encoder.PokemonEncoder import PokemonEncoder

from learning.LossFunctions.ContractiveLoss import ContrastiveLoss


pokemon_model = PokemonEncoder(device).to(device)
move_model = MoveEncoder(device).to(device)
# image_model.load_state_dict(torch.load('model/model_image_2023-07-07.pth'))
# caption_model.load_state_dict(torch.load('model/model_caption_2023-07-06.pth'))
pokemon_optimizer = torch.optim.Adam(pokemon_model.parameters(), lr=learning_rate)
move_optimizer = torch.optim.Adam(move_model.parameters(), lr=learning_rate)

loss_fn = ContrastiveLoss()

d:\Anaconda\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\tanaka\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [7]:
def train_loop(dataloader, pokemon_model: PokemonEncoder, move_model: MoveEncoder, loss_fn, poke_opt, move_opt):
    size = len(dataloader.dataset)
    for batch, (pokemon, move, label, _) in enumerate(dataloader):        
        # 予測と損失の計算
        label = label.to(device)
        pred = pokemon_model(pokemon)
        target = move_model(move)
        # ここ不安
        loss = loss_fn(pred, target, label)

        # バックプロパゲーション
        poke_opt.zero_grad()
        move_opt.zero_grad()
        loss.backward()
        poke_opt.step()
        move_opt.step()

        if batch % 30 == 0:
            loss, current = loss.item() / len(pokemon), batch * len(pokemon)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        if batch % 1000 == 0:
            # 現在の日付を取得します
            now = datetime.now()

            # YYYY-MM-DD形式で日付を出力します
            formatted_date = now.strftime("%Y-%m-%d")
            torch.save(pokemon_model.state_dict(), f'pokemon_model_{formatted_date}.pth')
            torch.save(move_model.state_dict(), f'move_model_{formatted_date}.pth')

def test_loop(dataloader, pokemon_model: PokemonEncoder, move_model: MoveEncoder, loss_fn):
    size = len(dataloader.dataset)
    test_loss = 0

    with torch.no_grad():
        for (pokemon, move, label, _) in dataloader:
            label = label.to(device)
            pred = pokemon_model(pokemon)
            target = move_model(move)
            loss = loss_fn(pred, target, label).mean()
            test_loss += loss.item()

    test_loss /= size
    print(f"Avg loss: {test_loss:>8f} \n")

In [8]:
print("start")
for t in range(epochs):
    print(f"Epoch {t+1}\-------------------------------")
    train_loop(train_dataloader, pokemon_model, move_model, loss_fn, pokemon_optimizer, move_optimizer)
    test_loop(test_dataloader, pokemon_model, move_model, loss_fn)
print("Done!")

start
Epoch 1\-------------------------------


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'learning.Entity.Pokemon.Pokemon'>